In [1]:
import torch,os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

True
cuda:0


In [2]:
model_name = "Llama-2"
model_name_or_path = "meta-llama/Llama-2-7b-chat-hf"

In [3]:
lr = 2e-4
num_epochs = 5
batch_size = 4

# Model init

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline, AutoModelForSeq2SeqLM,LlamaTokenizerFast
from langchain.llms import HuggingFacePipeline
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,cache_dir="Thin/llms")
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,torch_dtype=torch.bfloat16,
                                              device_map="auto",cache_dir="Thin/llms")

/home/itp1hc/.conda/envs/thinSA/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-28 05:13:45.250824: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 05:13:45.250923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 05:13:45.278808: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 05:13:45.353864: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow bi

# Run experiments

# Dataset

In [37]:
path_train = 'dataset/Homomexican/track_2_train.csv'
path_dev = 'dataset/Homomexican/track_2_dev.csv'
path_test = 'dataset/Homomexican/track_2_public_test.csv'
path_output = "./Output/SA/HSA/"

import pandas as pd
data_train = pd.read_csv(path_train)
#data_train.reset_index(drop=True, inplace=False)
#id_train = data_train['id_EXIST'].tolist()

data_test= pd.read_csv(path_test)
#data_test.reset_index(drop=True, inplace=False)
id_test = data_test["sub_id"].tolist()

data_dev= pd.read_csv(path_dev)
#data_dev.reset_index(drop=True, inplace=False)
#id_dev = data_dev["id_EXIST"].tolist()

data_test.head()

,Unnamed: 0,sub_id,content
0,0,0_Track2,Es más chida la gente transfóbica que lo acept...
1,1,1_Track2,A él ni lo menciones porque es joto. Jajaja
2,2,2_Track2,Tenemos a la directiva más puta marica del fút...
3,3,3_Track2,@ThierryHenry marica! ¿Ya olvidaste como celeb...
4,4,4_Track2,"@EPN Chinga a tu madre maricón de mierda, tu..."


# Preprocessing tweet

In [6]:
import re
def normalize_hashtag(doc):
    new_text = re.sub(r"#\w+\s*", "", doc, flags=re.MULTILINE) # first remove all hashtag and followed word in tweet
    pattern = r"(?=[A-Z 0-9])"
    for hashtag in re.findall(r"(?:\#)\S+", doc): #for each hashtag in text
        if not hashtag.isupper(): # if that hashtag is not fully upper case
            split = re.split(pattern, hashtag) #split that hashtag by upper case letter, example: GamerGirl -> Gamer Girl
            hashtag = ' '.join(str(x) for x in split if len(str(x))>1) #some bad case: AMDRedTeam -> A M D Red Team. The only thing we could do is removing any single letter in that text
        new_word = re.sub(r'#',"", hashtag, flags=re.MULTILINE) #remove "#" in text
        new_text = f"{new_text} {new_word}" # adding processed hashtag back to tweet
    return new_text

data_train['content'] = data_train['content'].apply(normalize_hashtag)
data_dev['content'] = data_dev['content'].apply(normalize_hashtag)
data_test['content'] = data_test['content'].apply(normalize_hashtag)

# preprocessing label

In [7]:
data_train['label'] = data_train[data_train.columns[2:]].values.tolist()
data_dev['label'] = data_dev[data_dev.columns[1:]].values.tolist()
def id2label(id_):
    label = ['LES','GAY','BI','TRAN','OTHER','NOT RELATED']
    out_label = []
    for index, i in enumerate(id_):
        if i == 1:
            out_label.append(label[index])
    return ', '.join(out_label)

data_train['label'] = data_train['label'].apply(id2label)

In [8]:
#Instruction + output prompt contain all task
import re
def create_instruction_input_output(df):
    input_text = []
    output_text = []
    inputs = df["content"].tolist()
    output = df["label"].tolist()
    for index, item in enumerate(inputs):     
        item = re.sub(r"\s+", " ", item) #replace all type of white space by a single space   
        
        prompt_new = f"""[INST]
Predict one or more labels of a tweet: "{item}"
## if the tweet contain hate speech directed at homosexual people who identify as female, output "LES"
## if the tweet contain hate speech directed at homosexual people who identify as male, output "GAY".
## if the tweet directed at people who attracted to more than one gender, output "BI".
## if the tweet against transgender, output "TRAN".
## if the tweet against other sexual and gender minorities, output "OTHER".
## if the tweet are not related in any way, output "NOT RELATED". OUTPUT only the labels, nothing else.
Answer: [/INST]"""     
        input_text.append(prompt_new)
        output_text.append(output[index])
    print(len(input_text),len(output_text))
    return input_text,output_text

In [9]:
# Create instruction input ouput for each task
input_train,output_train = create_instruction_input_output(data_train)
print(input_train[3])
print(output_train[3])

1071 1071
[INST]
Predict one or more labels of a tweet: "No llore jota"
## if the tweet contain hate speech directed at homosexual people who identify as female, output "LES"
## if the tweet contain hate speech directed at homosexual people who identify as male, output "GAY".
## if the tweet directed at people who attracted to more than one gender, output "BI".
## if the tweet against transgender, output "TRAN".
## if the tweet against other sexual and gender minorities, output "OTHER".
## if the tweet are not related in any way, output "NOT RELATED". OUTPUT only the labels, nothing else.
Answer: [/INST]
GAY


In [10]:
train_df = pd.DataFrame(list(zip(input_train, output_train)), columns =['text', 'label'])
# # removing rows that contain "UNKNOWN" OR ""-""
# train_df = train_df[(train_df["label"] != "-") & (train_df["label"] != "UNKNOWN")]
train_df.head(6)

,text,label
0,[INST]\nPredict one or more labels of a tweet:...,"LES, GAY"
1,[INST]\nPredict one or more labels of a tweet:...,GAY
2,[INST]\nPredict one or more labels of a tweet:...,"GAY, OTHER"
3,[INST]\nPredict one or more labels of a tweet:...,GAY
4,[INST]\nPredict one or more labels of a tweet:...,GAY
5,[INST]\nPredict one or more labels of a tweet:...,GAY


In [11]:
# create a dictionary datatype contain train, val set
from datasets import Dataset, DatasetDict

d1s = Dataset.from_pandas(train_df)

dataset = DatasetDict()
dataset['train'] = d1s
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1071
    })
})


# MODEL

In [12]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [13]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [14]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8, # Lora attention dimension.
        lora_alpha=16, # the alpha parameter for Lora scaling.
        lora_dropout=0.05, # the dropout probability for Lora layers.
        bias="none",
        target_modules=[
            #"embed_tokens",
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
            #"lm_head",
        ]
)

model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

trainable params: 19988480 || all params: 6758404096 || trainable%: 0.2957573965106688


In [15]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

# data preprocessing
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
# max_length = max([len(tokenizer(review)["input_ids"]) for review in df1["x_input"][100:200].tolist()])
# print(max_length)
max_length = 600

In [16]:
def preprocess_function(examples):
    batch_size = len(examples["text"])
    inputs = [item + " " for item in examples["text"]]
    targets = examples["label"]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets, add_special_tokens=False)  # don't add bos token because we concatenate with inputs
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]

        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (max_length - len(sample_input_ids)) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs["attention_mask"][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset[f"train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

#train_dataset = processed_datasets["train"]
train_dataset = processed_datasets[f"train"]
#eval_dataset = processed_datasets["test"]


train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
#eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)


Running tokenizer on dataset: 100%|██████████| 1071/1071 [00:00<00:00, 3176.35 examples/s]


In [17]:
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

# training and evaluation
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        #         print(batch)
        #         print(batch["input_ids"].shape)
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    # model.eval()
    # eval_loss = 0
    # eval_preds = []
#     for step, batch in enumerate(tqdm(eval_dataloader)):
#         batch = {k: v.to(device) for k, v in batch.items()}
#         with torch.no_grad():
#             outputs = model(**batch)
#         loss = outputs.loss
#         eval_loss += loss.detach().float()
#         eval_preds.extend(
#             tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
#         )

#     eval_epoch_loss = eval_loss / len(eval_dataloader)
#     eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=}") #{eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 268/268 [02:42<00:00,  1.65it/s]


epoch=0: train_ppl=tensor(1.2316, device='cuda:0') train_epoch_loss=tensor(0.2083, device='cuda:0')


100%|██████████| 268/268 [02:42<00:00,  1.65it/s]


epoch=1: train_ppl=tensor(1.0955, device='cuda:0') train_epoch_loss=tensor(0.0912, device='cuda:0')


100%|██████████| 268/268 [02:42<00:00,  1.65it/s]


epoch=2: train_ppl=tensor(1.0590, device='cuda:0') train_epoch_loss=tensor(0.0573, device='cuda:0')


100%|██████████| 268/268 [02:42<00:00,  1.65it/s]


epoch=3: train_ppl=tensor(1.0339, device='cuda:0') train_epoch_loss=tensor(0.0333, device='cuda:0')


100%|██████████| 268/268 [02:42<00:00,  1.65it/s]

epoch=4: train_ppl=tensor(1.0161, device='cuda:0') train_epoch_loss=tensor(0.0160, device='cuda:0')


# Prediction

In [18]:
def evaluate_model(sample,max_len_input=max_length):
    inputs = tokenizer(sample, return_tensors="pt").to('cuda')
    generation_config  = GenerationConfig(
    do_sample=True,
    max_new_tokens=128,
    top_k=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    penalty_alpha = 0.6,
    return_full_text=False,
    )
    
    outputs = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    output = output[0].split("[/INST]")[1].strip()
    return output

In [19]:
#Instruction + output prompt contain all task
import re
def create_instruction_input(df):
    input_text = []
    output_text = []
    inputs = df["content"].tolist()
    for index, item in enumerate(inputs):     
        item = re.sub(r"\s+", " ", item) #replace all type of white space by a single space   
        
        prompt_new = f"""[INST]
Predict one or more labels of a tweet: "{item}"
## if the tweet contain hate speech directed at homosexual people who identify as female, output "LES"
## if the tweet contain hate speech directed at homosexual people who identify as male, output "GAY".
## if the tweet directed at people who attracted to more than one gender, output "BI".
## if the tweet against transgender, output "TRAN".
## if the tweet against other sexual and gender minorities, output "OTHER".
## if the tweet are not related in any way, output "NOT RELATED". OUTPUT only the labels, nothing else.
Answer: [/INST]"""
        input_text.append(prompt_new)
    print(len(input_text))
    return input_text

## PREDICTION ON TEST

In [20]:
test_input = create_instruction_input(data_test)

268


In [21]:
from tqdm import tqdm

y_pred = []

#predict one batch of datadev at a time
for sample in tqdm(test_input):
    pred = evaluate_model(sample)
    y_pred.append(pred)

print(len(y_pred))
print(y_pred[:5])

100%|██████████| 268/268 [01:09<00:00,  3.85it/s]

268
['TRAN', 'GAY', 'GAY', 'GAY', 'GAY']


In [22]:
df_test = pd.DataFrame({'sub_id': id_test,
                      'label': y_pred,})
df_test.head()

,sub_id,label
0,0_Track2,TRAN
1,1_Track2,GAY
2,2_Track2,GAY
3,3_Track2,GAY
4,4_Track2,GAY


In [36]:
data_train['label'].tolist().count('NOT RELATED')

0

In [23]:
def label2id(out_label):
    label = ['LES','GAY','BI','TRAN','OTHER','NOT RELATED']
    xlist = out_label.split(", ")
    out_id = [0, 0, 0, 0, 0, 0]
    for index, labe in enumerate(label):
        if labe in xlist:
            out_id[index] = 1
    return out_id

y_pred_pro = [label2id(i) for i in y_pred]

In [24]:
y_pred_pro[:5]

[[0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0]]

In [25]:
df_test = pd.DataFrame({'sub_id': id_test,
                      'L': [i[0] for i in y_pred_pro],
                       'G': [i[1] for i in y_pred_pro],
                       'B': [i[2] for i in y_pred_pro],
                       'T': [i[3] for i in y_pred_pro],
                       'O': [i[4] for i in y_pred_pro],
                       'NR': [i[5] for i in y_pred_pro]})
df_test.head()

,sub_id,L,G,B,T,O,NR
0,0_Track2,0,0,0,1,0,0
1,1_Track2,0,1,0,0,0,0
2,2_Track2,0,1,0,0,0,0
3,3_Track2,0,1,0,0,0,0
4,4_Track2,0,1,0,0,0,0


In [26]:
df_test.to_csv(f'homomex21test.csv')

## PREDICTION ON DEV

In [27]:
dev_input = create_instruction_input(data_dev)
dev_output = data_dev["label"].tolist()

862


In [28]:
y_pred_dev = []

#predict one batch of datadev at a time
for sample in tqdm(dev_input):
    pred = evaluate_model(sample)
    y_pred_dev.append(pred)

print(len(y_pred_dev))
print(y_pred_dev[:5])

100%|██████████| 862/862 [03:48<00:00,  3.78it/s]

862
['GAY', 'OTHER', 'GAY', 'GAY', 'GAY']


In [29]:
y_pred_dev_pro = [label2id(i) for i in y_pred_dev]

In [30]:
with open(f'Homomex21dev.txt', 'w') as f: 
    for word in y_pred_dev: 
        f.write(f'{word}\n') 

In [31]:
len(y_pred_dev_pro)

862

-----------------------------------------------------------------------------------------------------------------

In [32]:
from sklearn.metrics import *
print(classification_report(data_dev["label"].tolist(), y_pred_dev_pro))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96        72
           1       1.00      1.00      1.00       714
           2       1.00      1.00      1.00        10
           3       1.00      1.00      1.00        79
           4       0.95      0.91      0.93        64
           5       0.00      0.00      0.00         0

   micro avg       1.00      0.99      0.99       939
   macro avg       0.83      0.81      0.82       939
weighted avg       1.00      0.99      0.99       939
 samples avg       1.00      0.99      0.99       939



/home/itp1hc/.conda/envs/thinSA/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/itp1hc/.conda/envs/thinSA/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
